In [ ]:
import cv2
import numpy as np
import os
from multiprocessing import Pool, cpu_count

def find_bounding_box(image, threshold=10):
    height, width, _ = image.shape

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    
    _, binary_image = cv2.threshold(gray_image, threshold, 255, cv2.THRESH_BINARY)

    rows = np.any(binary_image, axis=1)
    cols = np.any(binary_image, axis=0)

    if np.any(rows) and np.any(cols):
        min_y = np.argmax(rows)
        max_y = len(rows) - 1 - np.argmax(rows[::-1])
        min_x = np.argmax(cols)
        max_x = len(cols) - 1 - np.argmax(cols[::-1])

        cropped_image = image[min_y:max_y+1, min_x:max_x+1]
    else:
        cropped_image = image

    return cropped_image

def process_image(filename, input_folder, output_folder):
    try:
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)

        if image is None:
            print(f"Image {filename} could not be read.")
            return

        cropped_image = find_bounding_box(image)

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cropped_image)
        print(f"Cropped image saved to {output_path}")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

def process_images_in_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    with Pool(cpu_count()) as pool:
        pool.starmap(process_image, [(filename, input_folder, output_folder) for filename in image_files])

input_folder = '/content/IGEM-IITR-Drylab/DATASET/REDDOTOA'
output_folder = '/content/DATASET_NEW/red-dot'
process_images_in_folder(input_folder, output_folder)